In [1]:
import numpy as np
from utils import DiscreteDistrib, discrete_wasserstein_distance

In [2]:
x1 = np.array([[0, 1], [2,3]])
n1 = x1.shape[0]
x2 =  np.ones((3, 2))
n2 = x2.shape[0]

w1 = np.ones(n1) / n1
w2 = np.ones(n2) / n2

In [3]:
P1 = DiscreteDistrib(w1, x1)
P2 = DiscreteDistrib(w2, x2)

dist, pi = discrete_wasserstein_distance(P1, P2, return_coupling=True)

In [5]:
dist, pi

(1.272019649514065,
 array([[0.16666667, 0.16666667, 0.16666667],
        [0.16666667, 0.16666667, 0.16666667]]))